# Tiled Kernel
The notebook follows the video from GPU Mode:

[Lecture 5: Going Further with CUDA for Python Programmers](https://www.youtube.com/watch?v=wVsR-YhaHlM)

In [ ]:
!pip install ninja

In [ ]:
!nvidia-smi

In [ ]:
def in_colab() -> bool:
    try:
        import google.colab
        return True
    except ImportError:
        return False


IN_COLAB = in_colab()

In [1]:
import numpy as np

In [2]:
np.random.seed(20250811)

a = np.random.rand(1024, 512)
b = np.random.rand(512, 128)
ab = a @ b

In [3]:
from dataclasses import dataclass

@dataclass
class Dim3:
    x: int = 0
    y: int = 0
    z: int = 0

# Dim3 = namedtuple("Dim3", ["x", "y", "z"])

In [ ]:

TW = 32
TS = 32
block_size = Dim3(
    x=np.ceil(b.shape[1] / TS),
    y=np.ceil(a.shape[0] / TS),
)


async def tiled_kernel(block_dim: Dim3, thread_dim: Dim3, thread_size: Dim3, m, n, res, r, c, k, mem, signalb):
    """

    """
    bx = block_dim.x * thread_size.x
    tx = thread_dim.x
    cc = bx + tx
    by = block_dim.y * thread_size.y
    ty = thread_dim.y
    cr = by + ty
    for ph in range(np.ceil(k / TW)):
        for tw in range(TW):
            mem[ty + tw] = m[ph * TW + tw + cr * k]
            mem[TW * TW + tx + tw] = n[(ph *  TW + tw) * c + tx]
        signalb.wait()


